<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Quantitative Finance

# `numpy` package

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

## Monte Carlo Simulation

### Single Path

#### Python

In [ ]:
import math
import random

In [ ]:
S0 = 36
T = 1.0
r = 0.06
sigma = 0.2

In [ ]:
M = 100

In [ ]:
dt = T / M
dt

In [ ]:
path = [S0]

In [ ]:
%%time
for t in range(1, M + 1):
    path.append(path[t - 1] * math.exp(
        (r - sigma ** 2 / 2) * dt +
        sigma * math.sqrt(dt) * random.gauss(0, 1))
               )

In [ ]:
from pylab import plt
plt.style.use('seaborn')
%config InlineBackend.figure_format = 'svg'

In [ ]:
plt.plot(path);

#### `numpy`

In [ ]:
import numpy as np

In [ ]:
from numpy.random import default_rng

In [ ]:
rng = default_rng()

In [ ]:
rn = rng.standard_normal((M))

In [ ]:
rn[0] = 0

In [ ]:
%%time
path = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                   sigma * math.sqrt(dt) * rn).cumsum())

In [ ]:
path = np.concatenate(([S0], path))

In [ ]:
plt.plot(path);

### Multiple Paths

#### Python

In [ ]:
I = 100000

In [ ]:
paths = []

In [ ]:
%%time
for i in range(I):
    path = [S0]
    for t in range(1, M + 1):
        path.append(path[t - 1] * math.exp(
            (r - sigma ** 2 / 2) * dt +
            sigma * math.sqrt(dt) * random.gauss(0, 1))
               )
    paths.append(path)

In [ ]:
for i in range(25):
    plt.plot(paths[i]);

In [ ]:
ST = [path[-1] for path in paths]

In [ ]:
ST_ = sum(ST) / len(ST)

In [ ]:
ST_

In [ ]:
S0 * math.exp(r * T)

In [ ]:
import sys

In [ ]:
sum([sys.getsizeof(path) for path in paths])

In [ ]:
import pickle

In [ ]:
store = '/Users/yves/Temp/data/'
# store = ''  # to store the files in the working directory

In [ ]:
%time pickle.dump(paths, open(store + 'paths.pkl', 'wb'))

In [ ]:
%time p = pickle.load(open(store + 'paths.pkl', 'rb'))

In [ ]:
!ls -n $store/paths*

#### `numpy`

In [ ]:
%time rn = rng.standard_normal((M, I))

In [ ]:
rn = (rn - rn.mean()) / rn.std()

In [ ]:
%%time
paths = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                   sigma * math.sqrt(dt) * rn).cumsum(axis=0))
paths = np.concatenate(([S0 * np.ones(I)], paths), axis=0)

In [ ]:
plt.plot(paths[:, :25]);

In [ ]:
ST_ = paths[-1].mean()
ST_

In [ ]:
S0 * math.exp(r * T)

In [ ]:
sys.getsizeof(paths)

In [ ]:
%time np.save(store + 'paths', paths)

In [ ]:
%time p = np.load(store + 'paths.npy')

In [ ]:
!ls -n $store/paths*

In [ ]:
!rm $store/paths*

## Pricing Options

In [ ]:
M = 100
I = 200000

In [ ]:
dt = T / M
df = math.exp(-r * dt)

In [ ]:
def simulate_paths(M=M, I=I):
    dt = T / M
    rn = rng.standard_normal((M, I))
    rn = (rn - rn.mean()) / rn.std()
    paths = S0 * np.exp(((r - sigma ** 2 / 2) * dt +
                       sigma * math.sqrt(dt) * rn).cumsum(axis=0))
    paths = np.concatenate(([S0 * np.ones(I)], paths), axis=0)
    return paths

### European Options

In [ ]:
K = 40

In [ ]:
%time paths = simulate_paths()

In [ ]:
h = np.maximum(K - paths, 0)

In [ ]:
P0 = math.exp(-r * T) * h[-1].mean()

In [ ]:
P0

In [ ]:
# 3.844 (Longstaff-Schwartz 2001)

### American Options

In [ ]:
%%time
V = h[-1]
for t in range(M - 1, 1, -1):
    reg = np.polyfit(paths[t], df * V, deg=5) 
    C = np.polyval(reg, paths[t])
    V = np.where(h[t] > C, h[t], df * V)
P0 = df * C.mean()

In [ ]:
P0

In [ ]:
# 4.478 (Longstaff-Schwartz 2001)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="mailto:training@tpq.io">training@tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> 